In [2]:
# BASE
# ------------------------------------------------------
import numpy as np
import pandas as pd
import os
import gc
import warnings
from pathlib import Path
#!pip install xgboost
import sklearn
from sklearn import linear_model
from sklearn.linear_model  import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
#!pip install xgboost
import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, r2_score
pd.set_option('display.max_colwidth', None)
import statsmodels.api as sm

# DATA VISUALIZATION
# ------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

# CONFIGURATIONS
# ------------------------------------------------------
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.2f}'.format
warnings.filterwarnings('ignore')


In [3]:
folder = 'C:/Users/ylumady/Downloads/store-sales-time-series-forecasting/'

In [4]:
# Define Gloabl Path Variables
# ------------------------------------------------------
current_directory = os.getcwd()
parent_directory = os.path.abspath(os.path.join(current_directory, os.pardir))

print("Parent directory:", parent_directory)

DATA_PATH = Path(parent_directory)
RAW_PATH = DATA_PATH / 'raw'
TRAIN_PATH = RAW_PATH / 'train.csv'
TEST_PATH = RAW_PATH / 'test.csv'
DATA_PATH



Parent directory: c:\Users\ylumady\Documents\kaggle-store-sales


WindowsPath('c:/Users/ylumady/Documents/kaggle-store-sales')

In [5]:
# Downlaod data from kaggle
# ------------------------------------------------------
if 'data' not in os.listdir('..') or 'raw' not in os.listdir('../data'):
    print('\ndownloading kaggle data...')
    ! mkdir ../data
    ! mkdir ../data/raw
    ! mkdir ../data/processed
    ! kaggle competitions download -c store-sales-time-series-forecasting
    ! unzip store-sales-time-series-forecasting.zip
    ! mv *.csv ../data/raw
    ! rm store-sales-time-series-forecasting.zip
    ! mkdir ../data/submissions
    ! mv ../data/raw/*_submission* ../data/submissions/
else:
    print('kaggle data already downloaded in ../data')


downloading kaggle data...


The syntax of the command is incorrect.
The syntax of the command is incorrect.
The syntax of the command is incorrect.
'kaggle' is not recognized as an internal or external command,
operable program or batch file.
'unzip' is not recognized as an internal or external command,
operable program or batch file.
'mv' is not recognized as an internal or external command,
operable program or batch file.
'rm' is not recognized as an internal or external command,
operable program or batch file.
The syntax of the command is incorrect.
'mv' is not recognized as an internal or external command,
operable program or batch file.


In [6]:
# Import data
train = pd.read_csv(folder+"train.csv")
test = pd.read_csv(folder+"test.csv")
stores = pd.read_csv(folder+"stores.csv")
#sub = pd.read_csv("../input/store-sales-time-series-forecasting/sample_submission.csv")   
transactions = pd.read_csv(folder+"transactions.csv").sort_values(["store_nbr", "date"])


# Datetime
train["date"] = pd.to_datetime(train.date)
test["date"] = pd.to_datetime(test.date)
transactions["date"] = pd.to_datetime(transactions.date)

# Data types
train.onpromotion = train.onpromotion.astype("float16")
train.sales = train.sales.astype("float32")
stores.cluster = stores.cluster.astype("int8")

h=900

train.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.00,0.00
1,1,2013-01-01,1,BABY CARE,0.00,0.00
2,2,2013-01-01,1,BEAUTY,0.00,0.00
3,3,2013-01-01,1,BEVERAGES,0.00,0.00
4,4,2013-01-01,1,BOOKS,0.00,0.00


In [7]:
#Import holiday data
holidays = pd.read_csv(folder+"holidays_events.csv")
holidays["date"] = pd.to_datetime(holidays.date)
holidays


# holidays[holidays.type == "Holiday"]
# holidays[(holidays.type == "Holiday") & (holidays.transferred == True)]

# Transferred Holidays
tr1 = holidays[(holidays.type == "Holiday") & (holidays.transferred == True)].drop("transferred", axis = 1).reset_index(drop = True)
tr2 = holidays[(holidays.type == "Transfer")].drop("transferred", axis = 1).reset_index(drop = True)
tr = pd.concat([tr1,tr2], axis = 1)
tr = tr.iloc[:, [5,1,2,3,4]]

tr

holidays = holidays[(holidays.transferred == False) & (holidays.type != "Transfer")].drop("transferred", axis = 1)
holidays = holidays.append(tr).reset_index(drop = True)


# Additional Holidays
holidays["description"] = holidays["description"].str.replace("-", "").str.replace("+", "").str.replace('\d+', '')
holidays["type"] = np.where(holidays["type"] == "Additional", "Holiday", holidays["type"])

# Bridge Holidays
holidays["description"] = holidays["description"].str.replace("Puente ", "")
holidays["type"] = np.where(holidays["type"] == "Bridge", "Holiday", holidays["type"])

 
# Work Day Holidays, that is meant to payback the Bridge.
work_day = holidays[holidays.type == "Work Day"]  
holidays = holidays[holidays.type != "Work Day"]  


# Split

# Events are national
events = holidays[holidays.type == "Event"].drop(["type", "locale", "locale_name"], axis = 1).rename({"description":"events"}, axis = 1)

holidays = holidays[holidays.type != "Event"].drop("type", axis = 1)
regional = holidays[holidays.locale == "Regional"].rename({"locale_name":"state", "description":"holiday_regional"}, axis = 1).drop("locale", axis = 1).drop_duplicates()
national = holidays[holidays.locale == "National"].rename({"description":"holiday_national"}, axis = 1).drop(["locale", "locale_name"], axis = 1).drop_duplicates()
local = holidays[holidays.locale == "Local"].rename({"description":"holiday_local", "locale_name":"city"}, axis = 1).drop("locale", axis = 1).drop_duplicates()

test['test/train'] = 'test'
train['test/train'] = 'train'


d = pd.merge(train.append(test), stores)
d["store_nbr"] = d["store_nbr"].astype("int8")


# National Holidays & Events
#d = pd.merge(d, events, how = "left")
d = pd.merge(d, national, how = "left")
# Regional
d = pd.merge(d, regional, how = "left", on = ["date", "state"])
# Local
d = pd.merge(d, local, how = "left", on = ["date", "city"])


# Work Day: It will be removed when real work day colum created
d = pd.merge(d,  work_day[["date", "type"]].rename({"type":"IsWorkDay"}, axis = 1),how = "left")



# EVENTS
events["events"] =np.where(events.events.str.contains("futbol"), "Futbol", events.events)

def one_hot_encoder(df, nan_as_category=True):
    original_columns = list(df.columns)
    categorical_columns = df.select_dtypes(["category", "object"]).columns.tolist()
    # categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns=categorical_columns, dummy_na=nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    df.columns = df.columns.str.replace(" ", "_")
    return df, df.columns.tolist()

events, events_cat = one_hot_encoder(events, nan_as_category=False)
events["events_Dia_de_la_Madre"] = np.where(events.date == "2016-05-08", 1,events["events_Dia_de_la_Madre"])
events = events.drop(239)

d = pd.merge(d, events, how = "left")
d[events_cat] = d[events_cat].fillna(0)



# New features
d["holiday_national_binary"] = np.where(d.holiday_national.notnull(), 1, 0)
d["holiday_local_binary"] = np.where(d.holiday_local.notnull(), 1, 0)
d["holiday_regional_binary"] = np.where(d.holiday_regional.notnull(), 1, 0)

# 
d["national_independence"] = np.where(d.holiday_national.isin(['Batalla de Pichincha',  'Independencia de Cuenca', 'Independencia de Guayaquil', 'Independencia de Guayaquil', 'Primer Grito de Independencia']), 1, 0)
d["local_cantonizacio"] = np.where(d.holiday_local.str.contains("Cantonizacio"), 1, 0)
d["local_fundacion"] = np.where(d.holiday_local.str.contains("Fundacion"), 1, 0)
d["local_independencia"] = np.where(d.holiday_local.str.contains("Independencia"), 1, 0)


holidays, holidays_cat = one_hot_encoder(d[["holiday_national","holiday_regional","holiday_local"]], nan_as_category=False)
d = pd.concat([d.drop(["holiday_national","holiday_regional","holiday_local"], axis = 1),holidays], axis = 1)



he_cols = d.columns[d.columns.str.startswith("events")].tolist() + d.columns[d.columns.str.startswith("holiday")].tolist() + d.columns[d.columns.str.startswith("national")].tolist()+ d.columns[d.columns.str.startswith("local")].tolist()
d[he_cols] = d[he_cols].astype("int8")

d[["test/train","family", "city", "state", "type"]] = d[["test/train","family", "city", "state", "type"]].astype("category")

del holidays, holidays_cat, work_day, local, regional, national, events, events_cat, tr, tr1, tr2, he_cols
gc.collect()

0

In [8]:
# Inegrate Holidays data
# Time Related Features
def create_date_features(df):
    df['month'] = df.date.dt.month.astype("int8")
    df['day_of_month'] = df.date.dt.day.astype("int8")
    df['day_of_year'] = df.date.dt.dayofyear.astype("int16")
    #df['week_of_month'] = (df.date.apply(lambda d: (d.day-1) // 7 + 1)).astype("int8")
    df['week_of_month'] = ((df['day_of_month']-1) // 7 + 1).astype("int8")


    df['week_of_year'] = (df.date.dt.weekofyear).astype("int8")
    df['day_of_week'] = (df.date.dt.dayofweek + 1).astype("int8")
    df['year'] = df.date.dt.year.astype("int32")
    df["is_wknd"] = (df.date.dt.weekday // 4).astype("int8")
    df["quarter"] = df.date.dt.quarter.astype("int8")
    df['is_month_start'] = df.date.dt.is_month_start.astype("int8")
    df['is_month_end'] = df.date.dt.is_month_end.astype("int8")
    df['is_quarter_start'] = df.date.dt.is_quarter_start.astype("int8")
    df['is_quarter_end'] = df.date.dt.is_quarter_end.astype("int8")
    df['is_year_start'] = df.date.dt.is_year_start.astype("int8")
    df['is_year_end'] = df.date.dt.is_year_end.astype("int8")
    # 0: Winter - 1: Spring - 2: Summer - 3: Fall
    df["season"] = np.where(df.month.isin([12,1,2]), 0, 1)
    df["season"] = np.where(df.month.isin([6,7,8]), 2, df["season"])
    df["season"] = pd.Series(np.where(df.month.isin([9, 10, 11]), 3, df["season"])).astype("int8")
    return df
#d["date"] = pd.to_datetime(d.date, errors='coerce')
#d = d.dropna()
#d = get_date_data()

d = create_date_features(d)

In [9]:
d["workday"] = np.where((d.holiday_national_binary == 1) | (d.holiday_local_binary==1) | (d.holiday_regional_binary==1) | (d['day_of_week'].isin([6,7])), 0, 1)
d["workday"] = pd.Series(np.where(d.IsWorkDay.notnull(), 1, d["workday"])).astype("int8")
d.drop("IsWorkDay", axis = 1, inplace = True)

# Wages in the public sector are paid every two weeks on the 15 th and on the last day of the month. 
# Supermarket sales could be affected by this.
d["wageday"] = pd.Series(np.where((d['is_month_end'] == 1) | (d["day_of_month"] == 15), 1, 0)).astype("int8")

d.tail(15)
d.shape

(3029400, 79)

In [10]:
# Resample to reduce data for dev
dFull = d
d = dFull.sample(frac=0.1, replace=True, random_state=1)
d.shape

(302940, 79)

In [11]:
d.sales.value_counts()

0.00       94289
1.00       11666
2.00        8814
3.00        6969
4.00        5817
           ...  
7755.52        1
94.33          1
1004.89        1
374.87         1
795.14         1
Name: sales, Length: 49325, dtype: int64

In [12]:
# preform test train split
dataTrainTestHold = d.sort_values('date',ascending=False)

cols = ['test/train' ,'sales', 'family', 'date', 'month','day_of_month', 'week_of_year','day_of_week']
dataTrainTest = dataTrainTestHold[cols]




In [35]:
dataTrain = dataTrainTest[dataTrainTest['test/train'] ==  'train'].reset_index(drop=True)
dataTest = dataTrainTest[dataTrainTest['test/train'] ==  'test'].reset_index(drop=True)


train_ = (dataTrain)
test_ = (dataTest)



#dataTrain.drop('family', axis=1, inplace=True)

#train_, test_ = train_test_split(dataTrain.dropna().reset_index(drop=True), test_size=0.2, random_state=42)


In [14]:
300104/5

60020.8

In [15]:
300104-60020

240084

In [39]:
train_

,test/train,sales,family,date,month,day_of_month,week_of_year,day_of_week
0,train,7.00,LADIESWEAR,2017-08-15,8,15,33,2
1,train,13.00,HOME AND KITCHEN II,2017-08-15,8,15,33,2
2,train,0.00,BABY CARE,2017-08-15,8,15,33,2
3,train,1047.30,PRODUCE,2017-08-15,8,15,33,2
4,train,18.00,BEAUTY,2017-08-15,8,15,33,2
...,...,...,...,...,...,...,...,...
300099,train,0.00,POULTRY,2013-01-01,1,1,1,2
300100,train,0.00,PET SUPPLIES,2013-01-01,1,1,1,2
300101,train,0.00,DAIRY,2013-01-01,1,1,1,2
300102,train,0.00,DAIRY,2013-01-01,1,1,1,2


In [17]:
train_

,test/train,sales,family,date,month,day_of_month,week_of_year,day_of_week
0,train,7.00,LADIESWEAR,2017-08-15,8,15,33,2
1,train,13.00,HOME AND KITCHEN II,2017-08-15,8,15,33,2
2,train,0.00,BABY CARE,2017-08-15,8,15,33,2
3,train,1047.30,PRODUCE,2017-08-15,8,15,33,2
4,train,18.00,BEAUTY,2017-08-15,8,15,33,2
...,...,...,...,...,...,...,...,...
240079,train,267.57,POULTRY,2013-12-01,12,1,48,7
240080,train,111.00,GROCERY II,2013-12-01,12,1,48,7
240081,train,0.00,BABY CARE,2013-12-01,12,1,48,7
240082,train,1493.00,BEVERAGES,2013-12-01,12,1,48,7


In [18]:
len(test_.family.value_counts())

33

In [19]:
####testing models
#### back end forecasting
from pmdarima import auto_arima, ARIMA

from forecast_tools.model_selection import (rolling_forecast_origin, 
                                            cross_validation_score)

from forecast_tools.baseline import (Naive1, 
                                     SNaive,
                                     Drift,
                                     Average,
                                     EnsembleNaive,
                                     baseline_estimators)


In [89]:
train_2 = train_.drop(['family','month','day_of_month','week_of_year','day_of_week','test/train'],axis = 1)
test_2 = test_.drop(['family','month','day_of_month','week_of_year','day_of_week','test/train'],axis = 1)

train_2=train_2.groupby(['date']).sum('sales')
test_2=test_2.groupby(['date']).sum('sales')
test_2= train_2.iloc[1347:,:]
train_2= train_2.iloc[:1347,:]

In [90]:
#
model_naive_original = Naive1()
model_naive_original.fit(train_2['sales'])
model_snaive_original = SNaive(30)
model_snaive_original.fit(train_2['sales'])
model_drift_original = Drift()
model_drift_original.fit(train_2['sales'])


##preds
#
preds_naive_original = pd.DataFrame(model_naive_original.predict(horizon=test_2.shape[0]),index = test_2.index)
preds_snaive_original = pd.DataFrame(model_snaive_original.predict(horizon=test_2.shape[0]),index = test_2.index)
preds_drift_original = pd.DataFrame(model_drift_original.predict(horizon=test_2.shape[0]),index = test_2.index)

In [107]:
model_arima_original = auto_arima(train_2['sales'],  seasonal=True,m=7,
                    suppress_warnings=True, 
                     error_action='ignore')




In [92]:
test_2

,sales
date,
2016-09-12,88200.45
2016-09-13,53706.79
2016-09-14,70856.20
2016-09-15,69182.34
2016-09-16,94404.59
...,...
2017-08-11,72954.88
2017-08-12,104377.29
2017-08-13,103013.53


In [99]:

preds_arima_original = pd.DataFrame(model_arima_original.predict(n_periods=test_2.shape[0]).values,index = test_2.index)

In [100]:
preds_arima_original

,0
date,
2016-09-12,72384.88
2016-09-13,65890.11
2016-09-14,58101.00
2016-09-15,61188.19
2016-09-16,70259.51
...,...
2017-08-11,71094.36
2017-08-12,73613.38
2017-08-13,74593.41


In [101]:
preds_arima_original.columns = ['arima']
preds_naive_original.columns = ['naive']
preds_snaive_original.columns= ['snaive']
preds_drift_original.columns= ['drift']

In [102]:
preds = pd.concat([preds_arima_original, preds_naive_original,preds_snaive_original,preds_drift_original,], axis=1)
preds['actual'] = test_2.sales

In [103]:
preds

,arima,naive,snaive,drift,actual
date,,,,,
2016-09-12,72384.88,66864.45,73098.06,66913.46,88200.45
2016-09-13,65890.11,66864.45,53877.26,66962.48,53706.79
2016-09-14,58101.00,66864.45,70411.49,67011.50,70856.20
2016-09-15,61188.19,66864.45,46707.91,67060.52,69182.34
2016-09-16,70259.51,66864.45,71884.33,67109.54,94404.59
...,...,...,...,...,...
2017-08-11,71094.36,66864.45,70411.49,83187.57,72954.88
2017-08-12,73613.38,66864.45,46707.91,83236.59,104377.29
2017-08-13,74593.41,66864.45,71884.33,83285.61,103013.53


In [109]:
import plotly.graph_objects as go


trace1 = go.Scatter(x=preds.index, y=preds.arima, mode='lines', name='Arima')
trace2 = go.Scatter(x=preds.index, y=preds.naive, mode='lines', name='Naive')
trace3 = go.Scatter(x=preds.index, y=preds.snaive, mode='lines', name='Snaive')
trace4 = go.Scatter(x=preds.index, y=preds.drift, mode='lines', name='Drift')
trace5 = go.Scatter(x=preds.index, y=preds.actual, mode='lines', name='Actual')
data = [trace1, trace2, trace3,trace4,trace5]

layout = go.Layout(title='Predictions vs Actual', xaxis=dict(title='Date'), yaxis=dict(title='Sales'))

figure = go.Figure(data=data, layout=layout)
figure.show()

In [ ]:
trace5 = go.Scatter(x=train_2.index, y=train_2.sales, mode='lines', name='Actual')
data = [trace5]

layout = go.Layout(title='Predictions vs Actual', xaxis=dict(title='Date'), yaxis=dict(title='Sales'))

figure = go.Figure(data=data, layout=layout)
figure.show()

In [32]:
train_2

,sales
date,
2013-12-01,22615.90
2013-12-02,55557.32
2013-12-03,29953.46
2013-12-04,39892.89
2013-12-05,39958.71
...,...
2017-08-11,72954.88
2017-08-12,104377.29
2017-08-13,103013.53
